# notebook for create init and true test model

In [1]:
import numpy as np
import math

# grid
R_earth = 6371.0

rr1=5900 
rr2=6400
tt1=(30.0)/180*math.pi
tt2=(50.0)/180*math.pi
pp1=(15.0)/180*math.pi
pp2=(40.0)/180*math.pi



In [3]:
# read ak135 model
ak135 = np.loadtxt('ak135_model.txt')

(138, 4)


In [4]:
import h5py
import os

try:
    os.mkdir("2_models")
except:
    print("dir models exists")

for Ngrid in [41,61,81,121,161]:
    # model
    n_rtp = [Ngrid,Ngrid,Ngrid]
    dr = (rr2-rr1)/(n_rtp[0]-1)
    dt = (tt2-tt1)/(n_rtp[1]-1)
    dp = (pp2-pp1)/(n_rtp[2]-1)
    rr = np.array([rr1 + x*dr for x in range(n_rtp[0])])
    tt = np.array([tt1 + x*dt for x in range(n_rtp[1])])
    pp = np.array([pp1 + x*dp for x in range(n_rtp[2])])

    eta_init = np.zeros(n_rtp)
    xi_init  = np.zeros(n_rtp)
    zeta_init = np.zeros(n_rtp)
    fun_init = np.zeros(n_rtp)
    vel_init = np.zeros(n_rtp)

    for ir in range(n_rtp[0]):
        dep = R_earth - rr[ir]
        if dep < 0:
            vel = 5.8
        else:
            for i in range(ak135.shape[0]):
                if (dep > ak135[i,0] and dep <= ak135[i+1,0]):
                    vel = ak135[i,1] + (ak135[i+1,1]-ak135[i,1])/(ak135[i+1,0]-ak135[i,0])*(dep-ak135[i,0])
                    break

        for it in range(n_rtp[1]):
            for ip in range(n_rtp[2]):

                z = rr[ir] * math.sin(tt[it])
                x = rr[ir] * math.cos(tt[it]) * math.cos(pp[ip])
                y = rr[ir] * math.cos(tt[it]) * math.sin(pp[ip])
                
                vel_init[ir,it,ip] = vel

    # write out in hdf5 format

    fout_init = h5py.File('2_models/model_N%d_%d_%d.h5'%(n_rtp[0],n_rtp[1],n_rtp[2]), 'w')

    # write out the arrays eta_init, xi_init, zeta_init, fun_init, a_init, b_init, c_init, f_init
    fout_init.create_dataset('eta', data=eta_init)
    fout_init.create_dataset('xi', data=xi_init)
    fout_init.create_dataset('zeta', data=zeta_init)
    fout_init.create_dataset('vel', data=vel_init)

    fout_init.close()


dir models exists
